Cargo primero la informacion de las estaciones y el indicativo.

In [17]:
import pandas as pd

df_indicativos = pd.read_csv('estaciones_meteo.csv', sep=';', engine='python')

df_indicativos.drop(columns=['Unnamed: 6'], inplace=True)

df_indicativos.rename(columns={'INDICATIVO': 'Indicativo'}, inplace=True)

df_indicativos

,Indicativo,NOMBRE,PROVINCIA,ALTITUD,LATITUD,LONGITUD
0,1387,A CORUÑA,A CORUÑA,58,432157 N,082517 W
1,1387E,"A CORUÑA, ALVEDRO",A CORUÑA,98,431825 N,082219 W
2,1428,"SANTIAGO DE COMPOSTELA, LABACOLLA",A CORUÑA,370,425317 N,082438 W
3,8178D,"ALBACETE, OBS.",ALBACETE,674,390020 N,015144 W
4,8175,"ALBACETE, LOS LLANOS",ALBACETE,702,385715 N,015123 W
...,...,...,...,...,...,...
90,2422,VALLADOLID,VALLADOLID,735,413827 N,044516 W
91,2539,"VALLADOLID, VILLANUBLA",VALLADOLID,846,414243 N,045120 W
92,2614,ZAMORA,ZAMORA,656,413056 N,054407 W
93,9390,DAROCA I,ZARAGOZA,779,410652 N,012436 W


Aqui creo la secuencia de los meses y el bucle para que se vayan repitiendo los indicativos para que haya tantas filas como meses por indicativo.

In [28]:
import pandas as pd

indicativos_unicos = df_indicativos['INDICATIVO'].unique()

nombres_columnas = ['INDICATIVO', 'FECHA']
fecha_df = pd.DataFrame(columns=nombres_columnas)

fechas = pd.date_range(start='2016-01-01', end='2023-12-01', freq='MS')

fechas_formateadas = [fecha.strftime('%Y-%m') for fecha in fechas]

for indicativo in indicativos_unicos:
    for fecha in fechas_formateadas:
        fecha_df = pd.concat([fecha_df, pd.DataFrame({'INDICATIVO': [indicativo], 'FECHA': [fecha]})], ignore_index=True)

fecha_df['FECHA'] = pd.to_datetime(fecha_df['FECHA'])

fecha_df['FECHA'] = fecha_df['FECHA'].dt.strftime('%Y-%m')

fecha_df

,INDICATIVO,FECHA
0,1387,2016-01
1,1387,2016-02
2,1387,2016-03
3,1387,2016-04
4,1387,2016-05
...,...,...
9115,9434,2023-08
9116,9434,2023-09
9117,9434,2023-10
9118,9434,2023-11


Aqui uno los dos df y elimino la columnma indicativo para que ya se muestren los nombres de las ciudades. Creo que repito cosas, pero si uso 'fecha_df' se desmonta y no se por que. 

In [27]:
indicativos_unicos = df_indicativos['INDICATIVO'].unique()

nueva_df = pd.DataFrame(columns=nombres_columnas)


# Repetir cada indicativo para cada fecha y agregarlo al DataFrame
for indicativo in indicativos_unicos:
    for fecha in fechas:
        nueva_df = pd.concat([nueva_df, pd.DataFrame({'INDICATIVO': [indicativo], 'FECHA': [fecha]})], ignore_index=True)

# Fusionar con el DataFrame de estaciones para agregar la información
nueva_df = nueva_df.merge(df_indicativos, on='INDICATIVO', how='left')

# Eliminar la columna 'INDICATIVO' y 'Unnamed: 6'
nueva_df.drop(columns=['INDICATIVO', 'Unnamed: 6'], inplace=True)

# Reordenar las columnas para poner 'FECHA' al final
nueva_df = nueva_df[[c for c in nueva_df if c != 'FECHA'] + ['FECHA']]

nueva_df['FECHA'] = nueva_df['FECHA'].dt.strftime('%Y-%m')

nueva_df

,NOMBRE,PROVINCIA,ALTITUD,LATITUD,LONGITUD,FECHA
0,A CORUÑA,A CORUÑA,58,432157 N,082517 W,2016-01
1,A CORUÑA,A CORUÑA,58,432157 N,082517 W,2016-02
2,A CORUÑA,A CORUÑA,58,432157 N,082517 W,2016-03
3,A CORUÑA,A CORUÑA,58,432157 N,082517 W,2016-04
4,A CORUÑA,A CORUÑA,58,432157 N,082517 W,2016-05
...,...,...,...,...,...,...
9115,"ZARAGOZA, AEROPUERTO",ZARAGOZA,249,413938 N,010015 W,2023-08
9116,"ZARAGOZA, AEROPUERTO",ZARAGOZA,249,413938 N,010015 W,2023-09
9117,"ZARAGOZA, AEROPUERTO",ZARAGOZA,249,413938 N,010015 W,2023-10
9118,"ZARAGOZA, AEROPUERTO",ZARAGOZA,249,413938 N,010015 W,2023-11


Hasta aqui perfecto. Posiblemente me guarde las columnas de provincia, altitud, latitud y longitud para usarlas por separado, pero en el df base tienen que estar. 

Entonces ahora vienen los problemas. El codigo que te pongo es el que mas se parece a lo que quiero conseguir, pero ni de coña. Se carga la estructura de tantos indicativos por meses del bucle, pasa de 9120 filas a mas de 180.000 y crea un monton de NaN. En este no coge la informacion del df anterior, porque si lo ponia no conseguía montar el df, me decia o que no encontraba cosas, o que las longitudes de filas de los dos df no eran las mismas. Por eso probaba tambien crear el df de todas las caracteristicas por separado y luego hacer un merge con el df anterior.

In [21]:
import os

directorio_base = '/Users/angel/Desktop/selección_csv'

dataframes = []

caracteristicas = ['DIAS_GRA', 'DIAS_LLU', 'DIAS_NIE', 'DIAS_NUB', 'DIAS_SOL', 'EVAPORACIÓN', 'HUMEDAD', 
                   'INSOLACIÓN', 'INSOLACIÓN%', 'LLUVIA_MES', 'LLUVIA>1mm', 'LLUVIA>30mm', 'RADIACIÓN', 
                   'TEMP_10cm', 'TEMP_20cm', 'TEMP_50cm', 'TEMP_MAX', 'TEMP_MEDIA_MAX', 'TEMP_MEDIA_MIN',
                   'TEMP_MIN', 'TEMP<0', 'TEMP>30', 'VIENTO>55', 'VIENTO>91']  

for año in range(2016, 2024):
    carpeta_año = os.path.join(directorio_base, str(año))
    archivos_csv = [archivo for archivo in os.listdir(carpeta_año) if archivo.endswith('.csv')]
    
    for archivo_csv in archivos_csv:
        ruta_csv = os.path.join(carpeta_año, archivo_csv)
        
        df = pd.read_csv(ruta_csv, sep=';')
        
        indicativo = df.iloc[0, 1]
        
        df = df.melt(id_vars=['Indicativo'], var_name='Mes', value_name=archivo_csv.split('_')[1].split('.')[0])
        
        dataframes.append(df)

df_completo = pd.concat(dataframes, ignore_index=True)

columnas = ['Indicativo'] + caracteristicas
df_completo = df_completo.reindex(columns=columnas)

df_completo

,Indicativo,DIAS_GRA,DIAS_LLU,DIAS_NIE,DIAS_NUB,DIAS_SOL,EVAPORACIÓN,HUMEDAD,INSOLACIÓN,INSOLACIÓN%,...,TEMP_20cm,TEMP_50cm,TEMP_MAX,TEMP_MEDIA_MAX,TEMP_MEDIA_MIN,TEMP_MIN,TEMP<0,TEMP>30,VIENTO>55,VIENTO>91
0,1387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1387E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8178D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188612,9981A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188613,8368U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188614,3260B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188615,8414A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
